# Analisis Depresi Mahasiswa dengan Model Machine Learning

Notebook ini berisi proses lengkap untuk memuat data, melakukan pra-pemrosesan (preprocessing), melatih dua model machine learning (Linear Regression dan SGD Classifier), dan mengevaluasi performa keduanya untuk tugas klasifikasi.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputerimport numpy as np

## BAGIAN 1: MEMBACA DAN MEMPERSIAPKAN DATA

Pada bagian ini, kita akan:
1. Membaca dataset dari file CSV.
2. Membersihkan dan mengubah format data (misalnya, konversi kolom boolean/string menjadi numerik).
3. Melakukan rekayasa fitur sederhana (membuat `CGPA_Numeric`).
4. Memisahkan fitur (X) dan target (y).
5. Membuat pipeline preprocessing untuk menangani kolom numerik dan kategorikal.
6. Membagi data menjadi set pelatihan dan pengujian.

In [2]:
# Membaca data dari file CSV
try:
    df = pd.read_csv("Dataset.csv", sep=';')
except Exception as e:
    print(f"Error membaca file dengan separator ';': {e}")
    print("Mencoba membaca dengan separator ','...")
    df = pd.read_csv("Dataset.csv")

print("Data berhasil dimuat. Memulai persiapan data untuk Machine Learning...")

# Mendefinisikan variabel target yang akan diprediksi
TARGET = 'Depression'

# Membuat pemetaan untuk kolom CGPA
cgpa_map = {
    '0 - 1.99': 1.0,
    '2.00 - 2.49': 2.25,
    '2.50 - 2.99': 2.75,
    '3.00 - 3.49': 3.25,
    '3.50 - 4.00': 3.75
}

# Mengonversi semua kolom boolean/string boolean menjadi format numerik (1/0)
for col in ['Gender', 'MaritalStatus', 'Depression', 'Anxiety', 'PanicAttack', 'SeekTreatment']:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: 1 if str(x).strip().lower() in ['female', 'true', 'yes'] else 0)
        elif df[col].dtype == 'bool':
            df[col] = df[col].astype(int)

# Menambahkan kolom CGPA_Numeric ke dataframe
if 'CGPA' in df.columns:
    df['CGPA_Numeric'] = df['CGPA'].map(cgpa_map)
    df['CGPA_Numeric'].fillna(df['CGPA_Numeric'].mean(), inplace=True)
    df.drop(columns=['CGPA_Grade'], inplace=True, errors='ignore')

# Mendefinisikan fitur (X) dan target (y)
features_to_drop = ['Timestamp', TARGET, 'Anxiety', 'PanicAttack', 'CGPA'] 
X = df.drop(columns=features_to_drop, errors='ignore')
y = df[TARGET]

# Mengidentifikasi kolom kategorikal dan numerik untuk preprocessing
X['StudyYear'] = X['StudyYear'].astype(str)
categorical_features = ['Course', 'StudyYear']
numerical_features = ['Age', 'CGPA_Numeric']

# Membuat pipeline untuk fitur numerik yang menangani nilai kosong
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
# Membuat preprocessor untuk mentransformasi kolom
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' 
)

# Memisahkan data menjadi data latih dan data uji (80% latih, 20% uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nUkuran data latih (X_train): {X_train.shape}")
print(f"Ukuran data uji (X_test): {X_test.shape}")

Data berhasil dimuat. Memulai persiapan data untuk Machine Learning...


Ukuran data latih (X_train): (80, 8)
Ukuran data uji (X_test): (21, 8)


## BAGIAN 2: ALGORITMA 1 - LINEAR REGRESSION (UNTUK KLASIFIKASI)

Meskipun Regresi Linear pada dasarnya adalah model untuk regresi, kita dapat menggunakannya untuk klasifikasi biner dengan menerapkan *threshold* (ambang batas) pada outputnya. Di sini, kita akan menggunakan threshold 0.5: jika prediksi > 0.5, kelasnya adalah 1, dan sebaliknya.

In [3]:
print("--- Melatih Model: Linear Regression ---")

# Membuat pipeline: Preprocessing -> Model
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# Melatih model
lr_pipeline.fit(X_train, y_train)

# Membuat prediksi (hasilnya adalah nilai kontinu)
y_pred_lr_raw = lr_pipeline.predict(X_test)

# Mengonversi hasil prediksi menjadi kelas biner (0 atau 1) dengan threshold 0.5
y_pred_lr = (y_pred_lr_raw > 0.5).astype(int)

# Evaluasi model
mse = mean_squared_error(y_test, y_pred_lr_raw)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"\nMean Squared Error (MSE) dari prediksi mentah: {mse:.4f}")
print(f"Akurasi setelah thresholding (> 0.5): {accuracy_lr:.4f}")
print("\nLaporan Klasifikasi (Linear Regression):")
print(classification_report(y_test, y_pred_lr, zero_division=0))

--- Melatih Model: Linear Regression ---

Mean Squared Error (MSE) dari prediksi mentah: 0.2223
Akurasi setelah thresholding (> 0.5): 0.7143

Laporan Klasifikasi (Linear Regression):
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        15
           1       0.00      0.00      0.00         6

    accuracy                           0.71        21
   macro avg       0.36      0.50      0.42        21
weighted avg       0.51      0.71      0.60        21


## BAGIAN 3: ALGORITMA 2 - STOCHASTIC GRADIENT DESCENT (SGD) CLASSIFIER

SGD Classifier adalah model yang lebih cocok untuk tugas klasifikasi. Dengan mengatur `loss='log_loss'`, model ini akan bekerja seperti Regresi Logistik yang dioptimalkan menggunakan SGD. Model ini akan langsung menghasilkan prediksi kelas (0 atau 1).

In [4]:
print("--- Melatih Model: Stochastic Gradient Descent (SGD) Classifier ---")

# Membuat pipeline: Preprocessing -> Model
# loss='log_loss' membuat SGDClassifier bekerja seperti Regresi Logistik
sgd_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', SGDClassifier(loss='log_loss', random_state=42, max_iter=1000, tol=1e-3))])

# Melatih model
sgd_pipeline.fit(X_train, y_train)

# Membuat prediksi (hasilnya sudah dalam bentuk kelas 0 atau 1)
y_pred_sgd = sgd_pipeline.predict(X_test)

# Evaluasi model
accuracy_sgd = accuracy_score(y_test, y_pred_sgd)
print(f"\nAkurasi: {accuracy_sgd:.4f}")

print("\nLaporan Klasifikasi (SGD Classifier):")
print(classification_report(y_test, y_pred_sgd, zero_division=0))

--- Melatih Model: Stochastic Gradient Descent (SGD) Classifier ---

Akurasi: 0.7619

Laporan Klasifikasi (SGD Classifier):
              precision    recall  f1-score   support

           0       0.79      0.93      0.85        15
           1       0.67      0.33      0.44         6

    accuracy                           0.76        21
   macro avg       0.73      0.63      0.65        21
weighted avg       0.75      0.76      0.74        21


### Visualisasi Confusion Matrix untuk SGD Classifier

Confusion matrix membantu kita memahami performa model secara lebih rinci, dengan menunjukkan jumlah prediksi yang Benar Positif (TP), Benar Negatif (TN), Salah Positif (FP), dan Salah Negatif (FN).

In [5]:
print("\nConfusion Matrix (SGD Classifier):")
cm = confusion_matrix(y_test, y_pred_sgd)
# Baris di bawah ini akan menampilkan matriks sebagai teks jika ini adalah baris terakhir
cm 

# Kode plotting asli di-comment untuk menghindari output gambar
# plt.figure(figsize=(6,5))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#             xticklabels=['No Depression', 'Depression'], 
#             yticklabels=['No Depression', 'Depression'])
# plt.title('Confusion Matrix - SGD Classifier')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()


Confusion Matrix (SGD Classifier):


array([[14,  1],
       [ 4,  2]])